In [5]:
!git clone https://github.com/DeamonAmelie/heart

Cloning into 'heart'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 22 (delta 3), reused 15 (delta 2), pack-reused 0
Unpacking objects: 100% (22/22), done.
